<a href="https://colab.research.google.com/github/EdvinsVormsbehers/EDIBO_14/blob/main/Edv%C4%ABns_EDIBO14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Shows all data from albums table with GenreId from 5 to 15.

In [ ]:
USE music_shop;
SELECT *
FROM albums
WHERE AlbumId BETWEEN 5 AND 15;

Shows all data from invoices which has both arguments true.

In [ ]:
USE music_shop;
SELECT *
FROM invoices
WHERE (BillingCountry = 'Germany' AND BillingCity = 'Berlin');

Shows all data from invices which has both arguments true OR third argument true.

In [ ]:
USE music_shop;
SELECT *
FROM invoices
WHERE (BillingCountry = 'Germany' AND BillingCity = 'Berlin')
OR (CustomerId>50);

Shows all data which has at least one of the arguments true.

In [ ]:
USE music_shop;
SELECT *
FROM invoices
WHERE (BillingCountry = 'Germany' OR BillingCity = 'London');

Shows data from invoices.BillingCity and customers.LastName just for the CustomerID rows that matches in both invoices and customer tables.

In [ ]:
USE music_shop;
SELECT invoices.CustomerId, invoices.BillingCity, customers.LastName
FROM invoices
INNER JOIN customers
ON invoices.CustomerId = customers.CustomerId;

In [ ]:
USE music_shop;
SELECT invoices.CustomerId, invoices.BillingCity, customers.LastName
FROM invoices
LEFT JOIN customers
ON invoices.CustomerId = customers.CustomerId;

In [ ]:
USE music_shop;
SELECT invoices.CustomerId, invoices.BillingCity, customers.LastName
FROM invoices
RIGHT JOIN customers
ON invoices.CustomerId = customers.CustomerId;

Shows ArtistId data from two tables without duplicating the data.

In [ ]:
USE music_shop;
SELECT ArtistId 
FROM artists
UNION
SELECT ArtistId
FROM albums

Shows data from column FirstName and changed data from column city in new column Location.

In [ ]:
USE music_shop;
SELECT FirstName, CASE City
                 WHEN 'Calgary' THEN 'From Calgary'
                 WHEN 'Edmonton' THEN 'From Edmonton'
                 ELSE 'From Space'
                 END AS Location

FROM employees
ORDER BY Location;

Same construction method as above but uses <>= this time.

In [ ]:
USE music_shop;
SELECT CustomerId,
 CASE 
  WHEN Total < 2 THEN 'Small purchase'
  WHEN Total < 7 THEN 'Average'
  ELSE 'Good one'
  END Purchase_volume
FROM invoices
ORDER BY Purchase_volume;

Shows LastName, FirstName,Title data from employees table for entities where city=Calgary. Limit shows- how many entries will be shown.ORDER BY can be ASC (Ascending) or DESC (Descending) order.

In [ ]:
USE music_shop;
SELECT LastName, FirstName, Title
FROM employees
WHERE City = 'Calgary'
ORDER BY LastName
Limit 5;

Creates users table.

In [ ]:
USE music_shop;
CREATE TABLE users
(
    UserId INTEGER PRIMARY KEY AUTO_INCREMENT NOT NULL,
    UserName NVARCHAR(30)  NOT NULL,
    `User` BOOLEAN NULL COMMENT 'Simple user - only view',
    `Manager` BOOLEAN NULL COMMENT 'Advanced user - view and modify',
    `Admin` BOOLEAN NULL COMMENT 'Super user - view, modify, delete',
    EmployeeId INTEGER NULL,
    FOREIGN KEY (EmployeeId) REFERENCES employees (EmployeeId)
                ON DELETE NO ACTION ON UPDATE NO ACTION
);
CREATE INDEX IFK_UserEmployeeId ON employees (EmployeeId);

Add user data and then create alter table for media_types

In [ ]:
USE music_shop;
SELECT * FROM media_types;
ALTER TABLE media_types
ADD createdAt datetime NOT NULL DEFAULT current_timestamp;

ALTER TABLE media_types
ADD createdBy integer NOT NULL DEFAULT 1;

ALTER TABLE media_types
ADD CONSTRAINT IFK_MediaTypesCreateUserId
FOREIGN KEY (createdBy) REFERENCES users(UserId);

ALTER TABLE media_types
MODIFY createdBy integer NOT NULL;

SELECT * FROM media_types;

In [ ]:
USE music_shop;
DELIMITER $$
CREATE DEFINER = `root`@`localhost` PROCEDURE procedure2(IN inArgument1 int, IN inArgument2 varchar(30), IN inArgument3 datetime, IN inArgument4 bool, IN inCreatedBy int)
BEGIN
     DECLARE last_Id int;
     DECLARE variable1 varchar(20);
     DECLARE variable2 int;
     DECLARE done INT DEFAULT FALSE;
     DECLARE my_cur CURSOR FOR SELECT st1.some_field1, st2.some_field2
                              FROM some_table1 st1
                              JOIN some_table2 st2 ON st2.some_field=st1.some_field;
     DECLARE CONTINUE HANDLER FOR NOT FOUND SET done = TRUE;
     INSERT INTO some_table (  Id,      Field1,      Field2,      Field3,     Field4,          createdAt,   createdBy)
                     VALUES (NULL, inArgument1, inArgument2, inArgument3, inArgument4, CURRENT_TIMESTAMP, inCreatedBy);
     COMMIT;    
     SET last_Id=LAST_INSERT_ID();
     
     OPEN my_cur;
     myLoop: LOOP
          FETCH my_cur INTO variable1, variable2;
          IF done THEN
               LEAVE myLoop;
          END IF;
          CALL procedure2(variable1, variable2);
     END LOOP;
     CLOSE my_cur;
     COMMIT;    
END$$
DELIMITER ;

With this procedure can be returned select result as well as some amount or value.

In [ ]:
USE music_shop;
DELIMITER $$
CREATE DEFINER = `root`@`localhost` PROCEDURE `get_invoices_by_empl`(IN inEmplId INT, IN inYear varchar(4), IN inMonth varchar(2), OUT outTotal DEC(10,2))
BEGIN
   SET @filtr = concat('%',inYear,'-',inMonth,'-%');
   SELECT pkEmployeeId, pkCustomerId, pkInvoiceId, InvoiceDate, Total
		FROM employees em
		JOIN customers c ON em.pkEmployeeId=c.fkSupportRepId
		JOIN invoices inv ON c.pkCustomerId=inv.fkCustomerId
		WHERE em.pkEmployeeId=inEmplId
		AND InvoiceDate LIKE @filtr
		ORDER BY InvoiceDate ASC;
   SET outTotal=(SELECT sum(Total)
				FROM employees em
				JOIN customers c ON em.pkEmployeeId=c.fkSupportRepId
				JOIN invoices inv ON c.pkCustomerId=inv.fkCustomerId
				WHERE em.pkEmployeeId=inEmplId
				AND InvoiceDate LIKE @filtr
				ORDER BY InvoiceDate ASC);
END$$
DELIMITER ; 

#SET @totalToFill = 0;
#CALL get_inoices_by_empl(3,'2013','01',@totalToFill);
#SELECT @totalToFill;

Function counts total of invoice using historical prices.

In [ ]:
USE music_shop;
DELIMITER $$
CREATE DEFINER=`root`@`localhost` FUNCTION `InvoiceTotalAudit`(inInvoiceId int) RETURNS decimal(10,2)
    DETERMINISTIC
BEGIN
DECLARE AuditTotalVar decimal(10,2);
SELECT sum(ii.Quantity * ii.UnitPrice) as AuditTotal
FROM invoices i
	JOIN invoice_items ii ON ii.fkInvoiceId = i.pkInvoiceId
	JOIN tracks t ON t.pkTrackId = ii.fkTrackId
	JOIN media_types md ON md.pkMediaTypeId = t.fkMediaTypeId
	JOIN genres g ON g.pkGenreId = t.fkGenreId
	JOIN albums al ON al.pkAlbumId = t.fkAlbumId
	JOIN artists ar ON ar.pkArtistId = al.fkArtistId
	WHERE i.pkInvoiceId = inInvoiceId INTO @AuditTotalVar;
RETURN (@AuditTotalVar);
END$$
DELIMITER ;

In [ ]:
USE music_shop;
SELECT 'CREATE TRIGGER before_insert_Artists_only_unique_values' AS '';
DELIMITER $$
CREATE TRIGGER before_insert_Artists_only_unique_values
     BEFORE INSERT ON Artists FOR EACH ROW
     BEGIN
          IF NEW.Name IN (SELECT Name FROM Artists) THEN
               SIGNAL SQLSTATE '45000'
                    SET MESSAGE_TEXT = 'Iearakstu nevar izveidot - tāds izpildītājs jau eksistē!';
          END IF;
     END;$$
DELIMITER ;
#DROP TRIGGER before_insert_Artists_only_unique_values;